In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import datetime
import os
import pickle

# get_feature
功能 根据-v和-station文件在"日期"目录下生成"线路-feature.txt"文件，记录训练集基本特征，并拼接训练数据

In [1]:
def find_history_dist(num,is_rain,is_workday,is_peek,on,start_station,next_station):
    with open('./history/'+str(num)+'_'+str(is_rain)+'_'+str(is_workday)+'_'+str(is_peek)+'.txt', 'rb') as data_file:
        b = pickle.load(data_file).reset_index()
        # 	O_UP 	Source_Station 	Target_Station
        b = b[(b['O_UP']==on)&(b['Source_Station']==start_station)&(b['Target_Station']==next_station)]
        if b.shape[0] > 0:
            return b.iloc[0]
        else :
            dic = {'Distance':[-1],'Diff_Time':[-1]}
            x = pd.DataFrame(dic)
            return x.iloc[0]
        

def describe(x):
    if x=='多云':
        return 0
    elif x=='阴':
        return 1
    elif x=='晴':
        return 2
    elif x=='小雨':
        return 3
    else :
        return 4

def rain(d,n):
    if (int(d)>=3 or int(n)>=3):
        return 1
    else :
        return 0

weather = pd.read_csv("./data/weather.txt", sep=",")
weather['days'] = pd.to_datetime(weather['days'],format='%Y年%m月%d日')
# weather[(weather['days']=='2017-10-'+str(train.iloc[1]['O_TIME'].day))]
# weather['dws'].drop_duplicates()
weather['dws_onehot'] = weather['dws'].apply(describe)
weather['nws_onehot'] = weather['nws'].apply(describe)
weather['is_rain'] = weather.apply(lambda x: rain(x['dws_onehot'], x['nws_onehot']), axis=1)
weather


def get_feature(line,day):
    path = './' + day + '/' + str(line) + '-station.txt'
    path1 = './' + day + '/' + str(line) + '-v.txt';
    if os.path.exists(path) and os.path.exists(path1):
        data = pd.read_table(path, header=None, sep=',', encoding='utf-8')
        data.columns = ["O_UP", "Source_Station", "Target_Station", "Distance", 'Diff_Time']
        busmaxv = pd.read_table(path1, header=None, sep=',', encoding='utf-8')
        busmaxv.columns = ['O_TIME', "O_UP", "Source_Station", "Target_Station", "MaxV"]
        busmaxv['O_TIME'] = pd.to_datetime(busmaxv['O_TIME'],format='%Y-%m-%d %H:%M:%S')
        data['O_TIME'] = busmaxv['O_TIME']
        data['O_LINENO'] = line
        # 5 aver_v
        data['aver_v'] = 3600 * data['Distance'] / data['Diff_Time']
        # 6 hour
        data['hour'] = data['O_TIME'].apply(lambda x: x.hour)
        # 7 is_peek
        data['is_peek'] = data['hour'].apply(lambda x: 1 if ((x>=7 and x <9) or (x>=16 and x<19)) else 0)
        # 8 weekday
        data['weekday'] =  busmaxv.iloc[0]['O_TIME'].weekday()
        # 9 is_workday
        data['is_workday'] = data['weekday'].apply(lambda x: 0 if x>=5 else 1)
        weat = weather[(weather['days'] == '2017-10-' + day)].iloc[0]
        # 10-14 dws nws dts nts is_rain
        data['dws'] = weat['dws_onehot']
        data['nws'] = weat['nws_onehot']
        data['dts'] = weat['dts']
        data['nts'] = weat['nts']
        data['is_rain'] = weat['is_rain']
        # 15 max_v
        data['max_v'] = busmaxv['MaxV']

        # history
        # 16 aver_diff

        tmp = data.apply(lambda x: find_history_dist(line,x['is_rain'], x['is_workday'],x['is_peek'],x['O_UP'], x['Source_Station'], x['Target_Station']), axis=1).reset_index()

        data['h_aver_diff'] = tmp['Diff_Time']
        # 17 aver_d
        data['h_aver_d'] = tmp['Distance']
        # 18 aver_v
        def averV(d,n):
            if (int(d)==-1 or int(n)==-1):
#                 print(d,n)
                return -1
            else :
                return 3600 * d / n
        data['h_aver_v'] = data.apply(lambda x: averV(x['h_aver_d'],x['h_aver_diff']), axis=1)

    #     # 前一站平均速度 need read from day-station 当前数据的上一行
    #     h1 = find_history(is_rain, is_workday, is_peek, line, on, start_station - 1, next_station - 1)
    #     # 19 last_aver_v
    #     last_aver_v = h1['Distance'].mean() / h1['Diff_Time'].mean()

        # 20 is block up route

        # 21 dynamic

        with open('./' + day + '/' + str(line) + '-feature.txt', 'wb') as data_file:
            pickle.dump(data, data_file)

In [ ]:
for d in range(18, 19):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)

    train = pd.read_csv("./data/train201710"+day+".csv", sep=",")
    train['O_TIME'] = pd.to_datetime('2017-10-' + day + ' ' + train['O_TIME'], format='%Y-%m-%d %H:%M:%S')
    train = train.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x: x.sort_values(by='O_TIME', ascending=True))
    
    if not os.path.exists('./'+day+'/'):
        os.mkdir('./'+day+'/')
    line = train['O_LINENO'].drop_duplicates()
    print(day,line.shape)
    for i in range(line.shape[0]):
        l = line.iloc[i]
        buses = train[train['O_LINENO'] == l]
#         print(l, buses.shape)
        get_feature(l,day)
    print(day, "pre finished")

In [ ]:
#contact train
train_data = pd.DataFrame()
for d in range(9, 25):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)

    train = pd.read_csv("./data/train201710"+day+".csv", sep=",")
    #train['O_TIME'] = pd.to_datetime('2017-10-' + day + ' ' + train['O_TIME'], format='%Y-%m-%d %H:%M:%S')
    #train = train.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x: x.sort_values(by='O_TIME', ascending=True))

    line = train['O_LINENO'].drop_duplicates()
    print(line.shape)
    for i in range(line.shape[0]):
        l = line.iloc[i]
        #print(l)
        path = './'+day+'/'+str(l)+'-feature.txt'
        if os.path.exists(path):
            with open(path, 'rb') as data_file:
                data = pickle.load(data_file)
            train_data = pd.concat([train_data,data])
    print(day,"finished")
print(train_data.shape)
with open('./history/train.txt', 'wb') as data_file:
    pickle.dump(train_data, data_file)